In [72]:
import osmnx as ox
import geopandas as gpd
import requests
from osm2geojson import json2geojson
from shapely.geometry import shape
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# -------- Step 1: Get Poznań Boundary --------
# Retrieve the administrative boundary for Poznań from OSM
poznan_boundary = ox.geocode_to_gdf("Poznan, Poland")
poznan_boundary = poznan_boundary.to_crs(epsg=4326)
# We'll use the first (and typically only) geometry as our polygon
poznan_polygon = poznan_boundary.geometry.iloc[0]

In [77]:
help(poznan_polygon)


Help on Polygon in module shapely.geometry.polygon object:

class Polygon(shapely.geometry.base.BaseGeometry)
 |  Polygon(shell=None, holes=None)
 |
 |  A geometry type representing an area that is enclosed by a linear ring.
 |
 |  A polygon is a two-dimensional feature and has a non-zero area. It may
 |  have one or more negative-space "holes" which are also bounded by linear
 |  rings. If any rings cross each other, the feature is invalid and
 |  operations on it may fail.
 |
 |  Parameters
 |  ----------
 |  shell : sequence
 |      A sequence of (x, y [,z]) numeric coordinate pairs or triples, or
 |      an array-like with shape (N, 2) or (N, 3).
 |      Also can be a sequence of Point objects.
 |  holes : sequence
 |      A sequence of objects which satisfy the same requirements as the
 |      shell parameters above
 |
 |  Attributes
 |  ----------
 |  exterior : LinearRing
 |      The ring which bounds the positive space of the polygon.
 |  interiors : sequence
 |      A sequence

In [80]:
# Use the bounding box of Poznań for the initial Overpass query
minx, miny, maxx, maxy = poznan_boundary.total_bounds

query = f"""
[out:json][timeout:25];
(
  node["amenity"="parcel_locker"]({miny},{minx},{maxy},{maxx});
  way["amenity"="parcel_locker"]({miny},{minx},{maxy},{maxx});
  relation["amenity"="parcel_locker"]({miny},{minx},{maxy},{maxx});
);
out body;
>;
out skel qt;
"""

url = "http://overpass-api.de/api/interpreter"
response = requests.get(url, params={'data': query})
data = response.json()

In [86]:
# Convert Overpass JSON to GeoJSON using the osm2geojson library
geojson_data = json2geojson(data)

features_with_geom = []
for feat in geojson_data["features"]:
    geom_dict = feat.get("geometry")
    if geom_dict is not None:
        try:
            feat["geometry"] = shape(geom_dict)
            features_with_geom.append(feat)
        except Exception as e:
            # Skip features with problematic geometries
            continue

parcel_lockers_gdf = gpd.GeoDataFrame.from_features(features_with_geom, crs="EPSG:4326")
print("Parcel lockers before spatial filtering:", parcel_lockers_gdf.shape)

Parcel lockers before spatial filtering: (1141, 5)


In [87]:
parcel_lockers_gdf.head()

,geometry,type,id,tags,nodes
0,POINT (16.99446 52.38325),node,2116924371,"{'amenity': 'parcel_locker', 'brand': 'DPD Pic...",NaN
1,POINT (16.85641 52.48658),node,2541883437,"{'amenity': 'parcel_locker', 'brand': 'Paczkom...",NaN
2,POINT (17.06912 52.40216),node,2608474232,"{'amenity': 'parcel_locker', 'brand': 'Paczkom...",NaN
3,POINT (16.93112 52.36634),node,2778350540,"{'amenity': 'parcel_locker', 'brand': 'Paczkom...",NaN
4,POINT (16.78217 52.4779),node,3127120424,"{'amenity': 'parcel_locker', 'brand': 'Paczkom...",NaN
